# Random Forests
***

In this notebook we will approach the prediction problem using Random Forests. 

There is an introductory section on random forests, condensed from *An introduction to Statistical Learning* by G. James, D. Witten, T. Hastie, and R. Tibshirani.

In [5]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import sklearn.model_selection as skm
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
from sklearn.tree import (DecisionTreeClassifier as DTC,
                          DecisionTreeRegressor as DTR,
                          plot_tree,
                          export_text)
from sklearn.metrics import (accuracy_score,
                             log_loss)
from sklearn.ensemble import \
     (RandomForestRegressor as RF,
      GradientBoostingRegressor as GBR)

## Theory
***

### Regression Trees
***

We  discuss the process of building a regression tree. Roughly speaking, there are two steps.
1. We divide the predictor space - that is, the set of possible values for $X_1, X_2, \ldots, X_p$ - into $J$ distinct and non-overlapping regions (high-dimensional rectangles), $R_1, R_2, \ldots, R_J$.
2. For every observation that falls into the region $R_j$, we make the same prediction, which is simply the mean of the response values for the training observations in $R_j$.

The goal is to find boxes $R_1, \ldots, R_J$ that minimize the RSS, given by
$$
\sum_{j=1}^J \sum_{i \in R_j}\left(y_i-\hat{y}_{R_j}\right)^2,
$$
where $\hat{y}_{R_j}$ is the mean response for the training observations within the $j$ th box. 

<!-- We take a top-down, greedy approach that is known as recursive binary splitting. It begins at the top of the tree (at which point all observations belong to a single region) and then successively splits the predictor space; each split is indicated via two new branches further down on the tree. It is greedy because at each step of the tree-building process, the best split is made at that particular step, rather than looking ahead and picking a split that will lead to a better tree in some future step. -->

In detail, for any $j$ and $s$, we define the pair of half-planes
$$
R_1(j, s)=\left\{X \mid X_j<s\right\} \text { and } R_2(j, s)=\left\{X \mid X_j \geq s\right\}
$$
and we seek the value of $j$ and $s$ that minimize the equation
$$
\sum_{i: x_i \in R_1(j, s)}\left(y_i-\hat{y}_{R_1}\right)^2+\sum_{i: x_i \in R_2(j, s)}\left(y_i-\hat{y}_{R_2}\right)^2,
$$
where $\hat{y}_{R_1}$ is the mean response for the training observations in $R_1(j, s)$, and $\hat{y}_{R_2}$ is the mean response for the training observations in $R_2(j, s)$. Finding the values of $j$ and $s$ that minimize can be done quite quickly, especially when the number of features $p$ is not too large.

Next, we repeat the process, looking for the best predictor and best cutpoint in order to split the data further so as to minimize the RSS within each of the resulting regions. However, this time, instead of splitting the entire predictor space, we split one of the two previously identified regions. We now have three regions. Again, we look to split one of these three regions further, so as to minimize the RSS. The process continues until a stopping criterion is reached; for instance, we may continue until no region contains more than five observations.

The process described above is likely to overfit. Therefore, we grow a very large tree $T_0$, and prune it back in order to obtain a subtree that leads to the lowest test error rate. 

Cost complexity pruning - also known as weakest link pruning - gives us a way to do this. We consider a sequence of trees indexed by a nonnegative tuning parameter $\alpha$, for which there corresponds a subtree $T \subset T_0$ such that
$$
\sum_{m=1}^{|T|} \sum_{i: x_i \in R_m}\left(y_i-\hat{y}_{R_m}\right)^2+\alpha|T|
$$
is as small as possible. Here $|T|$ indicates the number of terminal nodes of the tree $T, R_m$ is the rectangle (i.e. the subset of predictor space) corresponding to the $m$ th terminal node, and $\hat{y}_{R_m}$ is the predicted response associated with $R_m$. As $\alpha$ increases, there is a price to pay for having a tree with many terminal nodes, and so the quantity above will tend to be minimized for a smaller subtree. It is reminiscent of the lasso.

**Algorithm: Building a Regression Tree:**
1. Use recursive binary splitting to grow a large tree on the training data, stopping only when each terminal node has fewer than some minimum number of observations.
2. Apply cost complexity pruning to the large tree in order to obtain a sequence of best subtrees, as a function of $\alpha$.
3. Use K-fold cross-validation to choose $\alpha$. That is, divide the training observations into $K$ folds. For each $k=1, \ldots, K$ :
    - (a) Repeat Steps 1 and 2 on all but the $k$ th fold of the training data.
    - (b) Evaluate the mean squared prediction error on the data in the left-out $k$ th fold, as a function of $\alpha$.
Average the results for each value of $\alpha$, and pick $\alpha$ to minimize the average error.
4. Return the subtree from Step 2 that corresponds to the chosen value of $\alpha$.

### Classification Trees
***
In the classification setting, RSS cannot be used as a criterion for making the binary splits. A natural alternative to RSS is the classification error rate, which is the fraction of the training observations in that region that do not belong to the most common class:
$$
E=1-\max _k\left(\hat{p}_{m k}\right) .
$$

Here $\hat{p}_{m k}$ represents the proportion of training observations in the $m$ th region that are from the $k$ th class. However, it turns out that classification error is not sufficiently sensitive for tree-growing.

An alternative is the Gini index is defined by
$$
G=\sum_{k=1}^K \hat{p}_{m k}\left(1-\hat{p}_{m k}\right),
$$
a measure of total variance across the $K$ classes. It is not hard to see a measure of total variance across the $K$ classes. It is not hard to see that the Gini index takes on a small value if all of the $\hat{p}_{m k}$ 's are close to zero or one. For this reason the Gini index is referred to as a measure of node purity.

## Import the data
***

In [6]:
train = pd.read_csv('data/processedTrain.csv')
test  = pd.read_csv('data/processedTest.csv')

drop_cols = ['Deck', 'Embarked', 'Family', 'Family_Size', 'Family_Size_Grouped', 'Survived',
             'Name', 'Parch', 'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Ticket', 'Title',
            'Ticket_Survival_Rate', 'Family_Survival_Rate', 'Ticket_Survival_Rate_NA', 'Family_Survival_Rate_NA']

y = train['Survived'].values
X = StandardScaler().fit_transform(train.drop(columns=drop_cols))
X_test  = StandardScaler().fit_transform(test.drop(columns=drop_cols))

X_train, X_cv, y_train, y_cv = skm.train_test_split(X, y, test_size=0.3, random_state=42)